In [1]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1583935105324_0018,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
import boto3

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkMagic Reference: https://github.com/jupyter-incubator/sparkmagic/blob/master/examples/Send%20local%20data%20to%20Spark.ipynb

In [2]:
import logging

logging.basicConfig(format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M')

console = logging.StreamHandler()

log_file=logging.FileHandler('/tmp/ml-lab-local.log', mode='a')


logger=logging.getLogger('ml-lab-local')
logger.addHandler(log_file)
logger.setLevel(logging.INFO)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from config import IDENTIFIER
from config import IG_BASE_URL

logger.info('IDENTIFIER: %s, IG_BASE_URL: %s' %(IDENTIFIER,IG_BASE_URL))

IDENTIFIER

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'demo2020'

In [4]:
from secret_manager import SecretManager
sm=SecretManager()
sm.get_secret()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from ig_client import IGClient
ig=IGClient(IG_BASE_URL,IDENTIFIER,sm.secret['ig_password'],sm.secret['ig_api_key'])
ig.login()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
session_info=ig.get_session_info()

logger.info('SESSION INFO:' %(session_info))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from lightstreamer_client import LightstreamerClient, LightstreamerSubscription

ls_client=LightstreamerClient(
    lightstreamer_url=session_info["lightstreamerEndpoint"],
    lightstreamer_username=IDENTIFIER,
    lightstreamer_password="CST-%s|XST-%s" %(ig.CST,ig.X_SECURITY_TOKEN)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
try:
    ls_client.connect()
except Exception as e:
    print(e)
    print("Unable to connect to Lightstreamer Server")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
subscription = LightstreamerSubscription(
    mode="MERGE",
    items=["MARKET:CS.D.GBPUSD.CFD.IP"],
    fields=["UPDATE_TIME","MID_OPEN","BID", "OFFER","CHANGE","CHANGE_PCT","HIGH","LOW"]
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# import logging.handlers

thread_log=logging.FileHandler('/tmp/ml-lab-threads.log', mode='a')

thread_logger=logging.getLogger('ml-lab-threads')
thread_logger.addHandler(thread_log)
thread_logger.setLevel(logging.INFO)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def thread_function(item):
    thread_logger.info(item)
        
# A simple function acting as a Subscription listener
def on_item_update(item):
    print(item)
    x = executor.submit(thread_function,item)

# Adding the "on_item_update" function to Subscription
subscription.addlistener(on_item_update)

# Registering the Subscription
sub_key = ls_client.subscribe(subscription)
logger.info("Subscription:%s" % sub_key)
thread_logger.info('OK')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
executor.shutdown(wait=False)
subscription.removelisteners()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
d=open('/tmp/ml-lab-local.log', 'r')
d.read()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'IDENTIFIER: demo2020, IG_BASE_URL: https://demo-api.ig.com/gateway/deal\nIDENTIFIER: demo2020, IG_BASE_URL: https://demo-api.ig.com/gateway/deal\nSESSION INFO:\nSESSION INFO:\nSubscription:1\nSubscription:1\nIDENTIFIER: demo2020, IG_BASE_URL: https://demo-api.ig.com/gateway/deal\nSESSION INFO:\nSubscription:1\nSubscription:2\nSubscription:3\nIDENTIFIER: demo2020, IG_BASE_URL: https://demo-api.ig.com/gateway/deal\nSESSION INFO:\nSubscription:1\n'

In [15]:
import os
os.listdir('/tmp/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['.ICE-unix', 'hsperfdata_root', 'lua_V4bBs8', 'root-public-key.pem', 'hsperfdata_hadoop', 'puppet_bigtop_file_merge', 'hsperfdata_hdfs', 'Jetty_localhost_33575_datanode____7at97e', 'hsperfdata_yarn', 'libleveldbjni-64-1-6123739768021477223.8', 'Jetty_0_0_0_0_8042_node____19tj0x', '.java_pid19032', '.java_pid26264', '.java_pid18948', '.java_pid19171', '.java_pid19689', '.java_pid19783', '.java_pid26477', 'messages.txt', 'ml-lab-ThreadPoolExecutor', 'ml-lab-ThreadPoolExecutor.log', 'ml-lab-local.log', 'ml-lab-threads.log', 'tmp30_9_27f']

In [16]:
d=open('/tmp/ml-lab-threads.log', 'r')
d.read()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:24:45', 'MID_OPEN': '1.29093', 'BID': '1.28439', 'OFFER': '1.28448', 'CHANGE': '-0.00649', 'CHANGE_PCT': '-0.5', 'HIGH': '1.29775', 'LOW': '1.28249'}}\n{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:24:45', 'MID_OPEN': '1.29093', 'BID': '1.28439', 'OFFER': '1.28448', 'CHANGE': '-0.00649', 'CHANGE_PCT': '-0.5', 'HIGH': '1.29775', 'LOW': '1.28249'}}\n{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:24:45', 'MID_OPEN': '1.29093', 'BID': '1.28439', 'OFFER': '1.28448', 'CHANGE': '-0.00649', 'CHANGE_PCT': '-0.5', 'HIGH': '1.29775', 'LOW': '1.28249'}}\n{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:24:45', 'MID_OPEN': '1.29093', 'BID': '1.28439', 'OFFER': '1.28448', 'CHANGE': '-0.00649', 'CHANGE_PCT': '-0.5', 'HIGH': '1.29775', 'LOW': '1.28249'}}\n{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:2